In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import utils

In [4]:
data = utils.load_data('../data/output.csv')

In [5]:
data.sample(5)

,Evento,Puerta,FechaHora,Rol
76041,Acceso concedido,RGD2-MOL3-IN-T5,2024/10/01 01:07:55,VIGILANCIA
19252,Acceso concedido,SD-VEHICULAR-SD-T9,2024/10/01 12:46:30,EGRESADO
8490,Acceso concedido,SD-VEHICULAR-SD-T10,2024/10/01 09:42:55,EGRESADO
65767,Acceso concedido,ML7-MOL3-OUT-T9,2024/10/01 14:13:43,ASEO
57828,Acceso concedido,LL2-MOL7-OUT-T2,2024/10/01 14:25:15,VIGILANCIA


In [6]:
data['custom_index'] = (data['Rol']).apply(utils.create_index)
data = data.sort_values(by='FechaHora').reset_index().drop(columns='index')
data.head(100000)

,Evento,Puerta,FechaHora,Rol,custom_index
0,Acceso concedido,SD-VEHICULAR-SD-T5,2024/10/01 00:00:05,ASEO,16546
1,Acceso concedido,ML7-MOL3-IN-T5,2024/10/01 00:00:09,VIGILANCIA,19813
2,Acceso concedido,CD9-MOL4-OUT-T7,2024/10/01 00:00:25,VIGILANCIA,19492
3,Acceso concedido,RGD6-MOL1-OUT-T11,2024/10/01 00:00:25,ASEO,16673
4,Acceso concedido,ML-VEHICULAR-ML-T7,2024/10/01 00:00:34,ASEO,16644
...,...,...,...,...,...
99995,Acceso concedido,ML-VEHICULAR-ML-T10,2024/10/01 23:59:54,PROFESOR,4056
99996,Acceso concedido,CD9-MOL4-OUT-T10,2024/10/01 23:59:54,PROFESOR,870
99997,Acceso concedido,SD11-MOL1-OUT-T7,2024/10/01 23:59:56,EMPLEADO,17815
99998,Acceso concedido,SD-VEHICULAR-SD-T6,2024/10/01 23:59:57,EMPLEADO,18018


In [13]:
import cv2
import numpy as np
import pytesseract

# Carga toda la carpeta de imagenes (assets) sin subcarpetas
images = utils.load_images('../assets')


for i, image in enumerate(images):

    # Convertir la imagen a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir los límites del color rojo en el espacio HSV
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear máscaras para ambos rangos de rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Unir las dos máscaras para cubrir todas las tonalidades de rojo
    mask = mask1 + mask2

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una lista para almacenar las coordenadas de los cuadrados
    square_coords = []

    # Filtrar solo los contornos que sean cuadrados y que tengan un área significativa
    for contour in contours:
        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar si el contorno tiene 4 lados (es un cuadrado o rectángulo)
        if len(approx) == 4 and cv2.contourArea(contour) > 100:  # Filtrar por área
            square_coords.append(approx)

            # Extraer la región del cuadrado detectado de la imagen original
            x, y, w, h = cv2.boundingRect(approx)
            square_roi = image[y:y + h, x:x + w]

            # Preprocesar la imagen antes de OCR (convertir a gris y aplicar umbral)
            gray_square = cv2.cvtColor(square_roi, cv2.COLOR_BGR2GRAY)

            # Binarizar la imagen (solo si es necesario)
            _, threshold_square = cv2.threshold(gray_square, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            text = pytesseract.image_to_string(threshold_square, config='--psm 7')  # psm 7 para una línea de texto

            # Decidir el color basado en el texto
            if "WAIRA" in text.upper():
                color = (0, 255, 0)  # Verde si el texto es "APPROVED"
            elif "ML-603" in text.upper():
                color = (0, 0, 255)  # Rojo si el texto es "REJECTED"
            else:
                color = (255, 255, 0)  # Amarillo para otros casos

            # Dibujar el cuadrado completo con el color basado en el texto
            cv2.fillPoly(image, [approx], color)

            # Poner el texto extraído en la imagen
            center_x = x + w // 2
            center_y = y + h // 2

            text_size = cv2.getTextSize(text.strip(), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
            text_width, text_height = text_size
            text_x = center_x - text_width // 2
            text_y = center_y + text_height // 2
            text_color = (255, 255, 255)
            cv2.putText(image, text.strip(), (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 2, text_color, 5, cv2.LINE_AA)

    # Guardar la imagen con los cuadrados detectados y coloreados
    cv2.imwrite(f'../assets/out/{i}.jpg', image)

## Procesado de datos Oferta de cursos

In [8]:
df = utils.load_data('../data/courses.csv')

In [9]:
df = df[df['Lugar']=='CAMPUS PRINCIPAL']
# Mantener solo los edificos usados para la investigación
df = df[df['Edificio'].str.contains(r'.*SD.*|.*\.centro.*|.*ML.*|.*LL.*', na=False)]
# Eliminar filas duplicadas
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4468 entries, 14 to 10438
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Curso                 4468 non-null   object
 1   Cupo                  4468 non-null   object
 2   NRC                   4468 non-null   int64 
 3   Sección               4468 non-null   object
 4   Créditos              4468 non-null   int64 
 5   Periodo               4468 non-null   int64 
 6   Parte de Periodo      4468 non-null   object
 7   Instructor Principal  4468 non-null   object
 8   Lugar                 4468 non-null   object
 9   Ofrecidos             4468 non-null   int64 
 10  Inscritos             4468 non-null   int64 
 11  Disponibles           4468 non-null   int64 
 12  Días                  4468 non-null   object
 13  Horas                 4468 non-null   object
 14  Salón                 4468 non-null   object
 15  Fecha inicio          4468 non-null   obj

In [10]:
# Mantener solo las columnas necesarias
df = df[['Curso','NRC','Periodo','Edificio','Salón','Días','Horas', 'Inscritos']]
# Eliminar cursos sin inscritos
df = df[df['Inscritos']>0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4457 entries, 14 to 10438
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Curso      4457 non-null   object
 1   NRC        4457 non-null   int64 
 2   Periodo    4457 non-null   int64 
 3   Edificio   4457 non-null   object
 4   Salón      4457 non-null   object
 5   Días       4457 non-null   object
 6   Horas      4457 non-null   object
 7   Inscritos  4457 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 313.4+ KB
